In [ ]:
import os
os.chdir("../")

In [ ]:
from src.constants import TARGET, INIT, DATA_PATH, NUMERIC_FEATURES, TEST_IDS,DATA_PATH


In [ ]:
import glob
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
import pickle as pkl
import numpy as np
import logging
import polars as pl

logger = logging.getLogger()
import sys
from tqdm import tqdm

logger.setLevel(logging.INFO)
from tqdm import tqdm

print(tf.__version__)
print(tf.config.list_physical_devices("GPU"))
if tf.test.is_gpu_available():
    print("TensorFlow está utilizando una GPU.")
    print("GPU utilizada:", tf.test.gpu_device_name())
else:
    print("TensorFlow no está utilizando una GPU.")

import matplotlib.pyplot as plt
import warnings
import joblib
import gc
import pickle as pkl
from itertools import groupby

In [ ]:
from pandas.core.arrays.timedeltas import timedelta
import matplotlib.pyplot as plt
from tensorflow.python.distribute.distribute_lib import def_function
from math import sqrt,exp,pi, gamma

## MODEL TARGET

It is used as the target whether the subject is sleeping or not (1 if they are and 0 if not). There are series with events that have 'step=null'. In these series, there are chunks of data where it is unknown where the previous or next event is. These data points are labeled as 'nan', and will be used only as features since the loss function uses a mask to compute the loss only on data points that do not have a 'nan' target.

Based on this target "sleep", events are constructed for evaluation and final submission using the 'get_event' function, which is being used in other notebooks.







In [ ]:
from src.params import (
    LEARNING_RATE,
    STEPS_PER_EPOCH,
    NUM_EPOCHS,
    WARMUP_STEPS,
    GPU_BATCH_SIZE,
    SAMPLE_NORMALIZE,
    DROP_INITIAL_DATE,
    OPTMIZER_BETA1,
    OPTMIZER_BETA2,
    OPTMIZER_EPSILON,
    ONLY_TEST,
    FOLD,
    TRAIN,
    PREDICT,
    CFG,
    DIM,
)
from src.constants import NUMERIC_FEATURES,SIGMA
train = TRAIN
predict = PREDICT

numeric_features = NUMERIC_FEATURES


In [ ]:
from src.utils import truncate_days,sample_normalize,drop_initial_date
from src.distributions import gauss, gauss_standard, student_t, lognormal_standard, lognorm,student_t
from src.data_processing import read_data

####  GET AND BUILD DATA: Convert binary target in smooth distribution to train a nn

In [ ]:

events = pd.read_csv(os.path.join(DATA_PATH,f"train_events.csv"))
events_check = pd.read_csv(os. path.join (DATA_PATH,f"train_events.csv")).dropna(
    subset="timestamp"
)
data = pd.read_parquet(
    os.path.join(DATA_PATH,f"data_train.parquet"),
    columns=["series_id", "step", "anglez", "enmo", "minute", "sine", "cosine", "sleep", "timestamp"],
).iloc[:1000]
data["timestamp"] = pd.to_datetime(data["timestamp"])

## train ids are mapped to int values so we need a dict to map back in the final part to comparare with the events dataset
dict_ids = pkl.load(open(f"{DATA_PATH}/dict_ids.pkl", "rb"))

targets, _, ids = joblib.load(f"{DATA_PATH}/train_data.pkl")
targets = {dict_ids[ids[i]]: targets[i] for i in range(len(targets))}

ids_ = np.load(f"{DATA_PATH}/ids.npy", allow_pickle=True).reshape(-1)
fold_ids = pkl.load(open(f"{DATA_PATH}/folds_ids.pkl", "rb"))
ids_=list(dict_ids.values())

In [ ]:
targets_events = targets
series_ = gauss(n=SIGMA, sigma=SIGMA * 0.15)
series_2 = gauss_standard(n=SIGMA, sigma=SIGMA * 0.15)
series_t = student_t(n=SIGMA, v=0.5)
series_lognorm = lognormal_standard()
plt.scatter(range(len(series_)), series_)
plt.scatter(range(len(series_t)), series_t, color="red")
plt.scatter(range(len(series_lognorm)), series_lognorm, color="orange")

In [ ]:
features, targets, descrp, steps_ = read_data(data, ids_,targets_events=targets_events,events_check=events_check,dict_ids=dict_ids)


In [ ]:
pkl.dump(features, open(os.path.join(DATA_PATH,"features.pkl"), "wb"))
pkl.dump(targets, open(os.path.join(DATA_PATH,"targets.pkl"), "wb"))
pkl.dump(steps_, open(os.path.join(DATA_PATH,"steps.pkl"), "wb"))